In [1]:
import os
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

# 사용할 모델 선택: 'local' 또는 'openai'
USE_LOCAL_MODEL = True  # False로 변경하면 OpenAI 사용

if USE_LOCAL_MODEL:
    try:
        import requests
        # Ollama 사용 (로컬 모델)
        OLLAMA_BASE_URL = "http://localhost:11434"
        OLLAMA_MODEL = "qwen3:4b"  # 또는 "qwen2.5", "gemma2" 등
        print(f"✅ 로컬 모델 사용: {OLLAMA_MODEL}")
        print(f"   Ollama 서버: {OLLAMA_BASE_URL}")
    except ImportError:
        print("⚠️ requests 라이브러리가 필요합니다: pip install requests")
else:
    from openai import OpenAI
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    print("✅ OpenAI API 사용: gpt-4o-mini")

✅ 로컬 모델 사용: qwen3:4b
   Ollama 서버: http://localhost:11434


In [2]:
# 카드 정보 입력 (예시)
card_info = {
    "type": "개발자",
    "attribute": "어둠",
    "rarity": "⭐",
    "attack": "1000",
    "health": "1000",
    "skill1_name": "디버깅",
    "skill1_description": "'몰입' 모드로 태세를 바꿉니다.",
    "skill2_name": "컴파일",
    "skill2_description": "모든 개발을 종료합니다.",
    "flavor_text": "세상 모든 0 과 1 은 나의 손에..."
}

print("📋 입력된 카드 정보:")
for key, value in card_info.items():
    print(f"  {key}: {value}")

📋 입력된 카드 정보:
  type: 개발자
  attribute: 어둠
  rarity: ⭐
  attack: 1000
  health: 1000
  skill1_name: 디버깅
  skill1_description: '몰입' 모드로 태세를 바꿉니다.
  skill2_name: 컴파일
  skill2_description: 모든 개발을 종료합니다.
  flavor_text: 세상 모든 0 과 1 은 나의 손에...


In [3]:
# 카드 이름 추천 함수

def generate_card_names_local(card_info, model=OLLAMA_MODEL):
    """로컬 모델(Ollama)을 사용하여 카드 이름 생성"""
    prompt = f"""다음 정보를 바탕으로 트레이딩 카드 게임에 적합한 카드 이름을 추천해주세요.

카드 정보:
- 타입: {card_info['type']}
- 속성: {card_info['attribute']}
- 등급: {card_info['rarity']}
- 공격력: {card_info['attack']}
- 체력: {card_info['health']}
- 스킬1: {card_info['skill1_name']} - {card_info['skill1_description']}
- 스킬2: {card_info['skill2_name']} - {card_info['skill2_description']}
- 플레이버 텍스트: {card_info['flavor_text']}

요구사항:
1. 카드의 타입, 속성, 스킬을 반영한 이름
2. 트레이딩 카드 게임에 적합한 멋진 이름
3. 한국어로 작성
4. 2-4단어로 구성된 간결한 이름
5. 카드의 특성과 분위기를 잘 표현하는 이름

카드 이름 3개를 추천해주세요. 각 이름은 한 줄에 하나씩, 번호 없이 제시해주세요."""
    
    response = requests.post(
        f"{OLLAMA_BASE_URL}/api/generate",
        json={
            "model": model,
            "prompt": prompt,
            "stream": False
        }
    )
    
    if response.status_code == 200:
        return response.json()["response"].strip()
    else:
        raise Exception(f"Ollama API 오류: {response.status_code} - {response.text}")


def generate_card_names_openai(card_info):
    """OpenAI API를 사용하여 카드 이름 생성"""
    prompt = f"""다음 정보를 바탕으로 트레이딩 카드 게임에 적합한 카드 이름을 추천해주세요.

카드 정보:
- 타입: {card_info['type']}
- 속성: {card_info['attribute']}
- 등급: {card_info['rarity']}
- 공격력: {card_info['attack']}
- 체력: {card_info['health']}
- 스킬1: {card_info['skill1_name']} - {card_info['skill1_description']}
- 스킬2: {card_info['skill2_name']} - {card_info['skill2_description']}
- 플레이버 텍스트: {card_info['flavor_text']}

요구사항:
1. 카드의 타입, 속성, 스킬을 반영한 이름
2. 트레이딩 카드 게임에 적합한 멋진 이름
3. 한국어로 작성
4. 2-4단어로 구성된 간결한 이름
5. 카드의 특성과 분위기를 잘 표현하는 이름

카드 이름 3개를 추천해주세요. 각 이름은 한 줄에 하나씩, 번호 없이 제시해주세요."""
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "당신은 트레이딩 카드 게임의 카드 이름을 추천하는 전문가입니다."
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        temperature=0.8,
        max_tokens=200
    )
    
    return response.choices[0].message.content.strip()


# 카드 이름 생성 실행
try:
    if USE_LOCAL_MODEL:
        print("\n🤖 카드 이름 추천 중... (로컬 모델: Ollama)")
        recommended_names = generate_card_names_local(card_info)
    else:
        print("\n🤖 카드 이름 추천 중... (OpenAI: gpt-4o-mini)")
        recommended_names = generate_card_names_openai(card_info)
    
    print("\n✅ 추천된 카드 이름:")
    print("=" * 50)
    print(recommended_names)
    print("=" * 50)
    
    # 이름들을 리스트로 분리
    name_list = [name.strip() for name in recommended_names.split('\n') if name.strip()]
    print(f"\n📝 추천된 이름 개수: {len(name_list)}개")
    
except Exception as e:
    print(f"❌ 오류 발생: {e}")
    if USE_LOCAL_MODEL:
        print("\n💡 Ollama 설치 및 실행 방법:")
        print("1. Ollama 설치: https://ollama.com/download")
        print(f"2. 모델 다운로드: ollama pull {OLLAMA_MODEL}")
        print("3. Ollama 서버 실행 확인: ollama serve")
    import traceback
    traceback.print_exc()


🤖 카드 이름 추천 중... (로컬 모델: Ollama)

✅ 추천된 카드 이름:
어둠 비트
디버그 어둠
0과 1의 손

📝 추천된 이름 개수: 3개
